In [1]:
import numpy as np
from tabulate import tabulate
from colorama import Fore, Style, init

# Inicializa o Colorama para Windows (se necessário)
init(autoreset=True)

def print_colored_table(table, pivot_row=None, pivot_col=None):
    # Função para imprimir o tableau com cores aplicadas ao pivot_row e pivot_col
    table_str = ""
    for i, row in enumerate(table):
        for j, elem in enumerate(row):
            if i == pivot_row and j == pivot_col:
                table_str += f"{Fore.RED}{elem:.2f}{Style.RESET_ALL}\t"
            elif i == pivot_row or j == pivot_col:
                table_str += f"{Fore.YELLOW}{elem:.2f}{Style.RESET_ALL}\t"
            else:
                table_str += f"{elem:.2f}\t"
        table_str += "\n"
    print(table_str)

def simplex(c, A, b):
    # Número de variáveis de decisão e restrições
    num_vars = len(c)
    num_constraints = len(b)

    # Monta o tableau inicial
    tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))

    # Configura a matriz das restrições (A) e dos termos independentes (b) no tableau
    tableau[:-1, :num_vars] = A
    tableau[:-1, num_vars:num_vars + num_constraints] = np.eye(num_constraints)
    tableau[:-1, -1] = b

    # Configura a função objetivo no tableau
    tableau[-1, :num_vars] = -np.array(c)  # Função objetivo negada para maximização

    print("Tabela inicial:")
    print_colored_table(tableau)

    # Executa o algoritmo Simplex
    step = 1
    while True:
        # Passo 1: Identificar a coluna de entrada (a variável que entra na base)
        pivot_col = np.argmin(tableau[-1, :-1])
        if tableau[-1, pivot_col] >= 0:
            # Se todos os coeficientes da linha da função objetivo são >= 0, encontramos a solução ótima
            break

        # Passo 2: Identificar a linha de saída (a variável que sai da base)
        with np.errstate(divide='ignore', invalid='ignore'):
            ratios = np.where(tableau[:-1, pivot_col] > 0, tableau[:-1, -1] / tableau[:-1, pivot_col], np.inf)
        pivot_row = np.argmin(ratios)

        # Se não existe um valor positivo, a solução é ilimitada
        if np.isinf(ratios[pivot_row]):
            print("A solução é ilimitada.")
            return None

        # Pivotamento
        pivot_element = tableau[pivot_row, pivot_col]
        print(f"\nEtapa {step}: Pivotamento")
        print(f"Elemento pivot: {pivot_element} (Linha {pivot_row}, Coluna {pivot_col})")

        # Normaliza a linha do pivot
        tableau[pivot_row] /= pivot_element
        print("Tabela após normalizar linha do pivot:")
        print_colored_table(tableau, pivot_row, pivot_col)

        # Atualiza as outras linhas para zerar a coluna do pivot
        for i in range(len(tableau)):
            if i != pivot_row:
                factor = tableau[i, pivot_col]
                tableau[i] -= factor * tableau[pivot_row]
                print(f"\nAtualizando linha {i} usando fator {factor}")
                print_colored_table(tableau, pivot_row, pivot_col)

        step += 1  # Incrementa o contador de etapas

        print(f"\nTabela após a Etapa {step - 1}:")
        print_colored_table(tableau)

    # Resultado: obter as variáveis básicas
    solution = np.zeros(num_vars)
    for i in range(num_vars):
        # Se a coluna correspondente é canônica, adiciona o valor à solução
        col = tableau[:-1, i]
        if np.count_nonzero(col) == 1 and np.any(col == 1):
            solution[i] = tableau[np.where(col == 1)[0][0], -1]

    # Lucro máximo (último elemento da última linha)
    max_profit = tableau[-1, -1]

    print("\nSolução ótima encontrada:")
    print("Valores das variáveis:", solution)
    print("Lucro total máximo =", max_profit)

    return solution, max_profit

# Dados do problema
c = [0.35, 0.12]  # Lucro por quilo de carne de boi e grão, respectivamente
A = [
    [1, 1],          # Restrição de peso total
    [0.2, 0.4],      # Restrição de volume total
    [1, 0],          # Disponibilidade máxima de carne de boi
    [0, 1]           # Disponibilidade máxima de grão
]
b = [160000, 70000, 85000, 100000]  # Restrições de capacidade e disponibilidade

# Executa o método Simplex
solution, max_profit = simplex(c, A, b)


Tabela inicial:
1.00	1.00	1.00	0.00	0.00	0.00	160000.00	
0.20	0.40	0.00	1.00	0.00	0.00	70000.00	
1.00	0.00	0.00	0.00	1.00	0.00	85000.00	
0.00	1.00	0.00	0.00	0.00	1.00	100000.00	
-0.35	-0.12	0.00	0.00	0.00	0.00	0.00	


Etapa 1: Pivotamento
Elemento pivot: 1.0 (Linha 2, Coluna 0)
Tabela após normalizar linha do pivot:
1.00	1.00	1.00	0.00	0.00	0.00	160000.00	
0.20	0.40	0.00	1.00	0.00	0.00	70000.00	
1.00	0.00	0.00	0.00	1.00	0.00	85000.00	
0.00	1.00	0.00	0.00	0.00	1.00	100000.00	
-0.35	-0.12	0.00	0.00	0.00	0.00	0.00	


Atualizando linha 0 usando fator 1.0
0.00	1.00	1.00	0.00	-1.00	0.00	75000.00	
0.20	0.40	0.00	1.00	0.00	0.00	70000.00	
1.00	0.00	0.00	0.00	1.00	0.00	85000.00	
0.00	1.00	0.00	0.00	0.00	1.00	100000.00	
-0.35	-0.12	0.00	0.00	0.00	0.00	0.00	


Atualizando linha 1 usando fator 0.2
0.00	1.00	1.00	0.00	-1.00	0.00	75000.00	
0.00	0.40	0.00	1.00	-0.20	0.00	53000.00	
1.00	0.00	0.00	0.00	1.00	0.00	85000.00	
0.00	1.00	0.00	0.00	0.00	1.00	100000.00	
-0.35	-0.12	0.00	0.00	0.00	0.00	0.00	


A

In [2]:
from scipy.optimize import linprog

# Coeficientes da função objetivo (note que 'linprog' minimiza, então usamos -c para maximizar)
c = [-0.35, -0.12]  # Lucro por quilo de carne de boi e grão, invertido para maximização

# Coeficientes das restrições
A = [
    [1, 1],       # Restrição de peso total (<= 160.000 kg)
    [0.2, 0.4],   # Restrição de volume total (<= 70.000 m³)
    [1, 0],       # Disponibilidade máxima de carne de boi (<= 85.000 kg)
    [0, 1]        # Disponibilidade máxima de grãos (<= 100.000 kg)
]

# Limites das restrições
b = [160000, 70000, 85000, 100000]  # Limites para as restrições acima

# Executa o método Simplex usando linprog
result = linprog(c, A_ub=A, b_ub=b, method="simplex")

# Exibe os resultados
if result.success:
    # Multiplica o valor da função objetivo por -1 para reverter a minimização para maximização
    max_profit = -result.fun
    solution = result.x
    print("Solução ótima encontrada:")
    print("Quantidade de carne de boi (x1):", solution[0])
    print("Quantidade de grãos (x2):", solution[1])
    print("Lucro total máximo:", max_profit)
else:
    print("O problema não tem solução ótima.")


C:\Users\Prodenge\AppData\Local\Temp\ipykernel_5612\2401266518.py:18: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, A_ub=A, b_ub=b, method="simplex")


Solução ótima encontrada:
Quantidade de carne de boi (x1): 85000.0
Quantidade de grãos (x2): 75000.0
Lucro total máximo: 38749.99999999999
